In [1]:
import torch
print(torch.cuda.is_available() , torch.__version__)

False 2.2.0+cpu


In [1]:
# %% test a specific model
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)

24-05-20 06:58:23|MOD:display     |: Model Specifics:
24-05-20 06:58:23|MOD:display     |: Start Process [Data] at Mon May 20 06:58:23 2024!


Use device name: NVIDIA GeForce RTX 4090
--Process Queue : Data + Fit + Test
--Model_name is set to gru_lgbm!
Callback : DynamicDataLink() , assign and unlink dynamic data in tra networks
Callback : ResetOptimizer(num_reset=2,trigger=40,recover_level=1.0,speedup2x=True) , reset optimizer on some epoch (can speedup scheduler)
Callback : CallbackTimer(verbosity=2) , record time cost of callback hooks
Callback : EarlyStoppage(patience=20) , stop fitting when validation score cease to improve
Callback : ValidationConverge(patience=5,eps=1e-05) , stop fitting when valid_score converge
Callback : EarlyExitRetrain(earliest=20,max_attempt=4,lr_multiplier=[1, 0.1, 10, 0.01, 100, 1]) , retrain with new lr if fitting stopped too early
Callback : NanLossRetrain(max_attempt=4) , retrain if fitting encounters nan loss
Callback : BatchDisplay(verbosity=2) , display batch progress bar
Callback : StatusDisplay(verbosity=2) , display epoch and event information
{'random_seed': None,
 'model_name': 'gru_

24-05-20 06:58:25|MOD:display     |: Finish Process [Data], Cost 2.2 Secs
24-05-20 06:58:25|MOD:display     |: Start Process [Fit] at Mon May 20 06:58:25 2024!


Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
score function of [spearman] calculated and success!
loss function of [pearson] calculated and success!


FirstBite Ep#  0 : loss  1.00267, train-0.00697, valid-0.05564, best-0.0556, lr1.3e-03
FirstBite Ep#  5 : loss  0.88727, train 0.12678, valid 0.13888, best 0.1401, lr2.5e-03
FirstBite Ep# 10 : loss  0.87124, train 0.14223, valid 0.15362, best 0.1536, lr1.9e-03
FirstBite Ep# 15 : loss  0.86148, train 0.15172, valid 0.16116, best 0.1612, lr1.0e-07
FirstBite Ep# 20 : loss  0.86000, train 0.15314, valid 0.15322, best 0.1612, lr9.4e-04
FirstBite Ep# 25 : loss  0.85520, train 0.15804, valid 0.15818, best 0.1612, lr3.1e-04
FirstBite Ep# 30 : loss  0.85394, train 0.15904, valid 0.15696, best 0.1612, lr1.6e-04
FirstBite Ep# 35 : loss  0.85347, train 0.15998, valid 0.15856, best 0.1612, lr3.1e-04
FirstBite Ep# 35 : loss  0.85347, train 0.15998, valid 0.15856, best 0.1612, lr3.1e-04, Next attempt goes!
Retrain#1 Ep#  0 : loss  1.01452, train-0.02067, valid-0.05125, best-0.0512, lr1.3e-04
Retrain#1 Ep#  5 : loss  0.93804, train 0.07483, valid 0.10295, best 0.1029, lr2.5e-04
Retrain#1 Ep# 10 : loss

KeyboardInterrupt: 

In [ ]:
from src.interface import ModelTrainer
ModelTrainer.main(stage = 0 , resume = 0 , checkname= 1)


In [ ]:
# prepare data
from src.interface import DataModule
DataModule.prepare_data()